# Perplexity playbook

<p>
Mal Minhas, v0.6<br>
09.11.24
</p>
<p>
<h4>Versions</h4>
<ul>
<li><b>v0.3</b>: 15.01.24. First version</li>
<li><b>v0.4</b>: 02.06.24. Updated for latest Perplexity API</li>
<li><b>v0.5</b>: 01.08.24. Updated again for latest Perplexity API and llama 3.1 support</li>
<li><b>v0.6</b>: 09.11.24. Updated to include support for retrieving citations</li>        
</ul>
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an OpenAI-compliant API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api` per the documetnation [here](https://docs.perplexity.ai/).  Perplexity regularly update the model support for this API and at the time of writing the latest and greatest supported model is `llama-3.1`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/guides/model-cards):
```
Perplexity Models
Model	Parameter Count	Context Length	Model Type
llama-3.1-sonar-small-128k-online	8B	127,072	Chat Completion
llama-3.1-sonar-large-128k-online	70B	127,072	Chat Completion
llama-3.1-sonar-huge-128k-online	405B	127,072	Chat Completion
llama-3.1-sonar-small-128k-chat	8B	127,072	Chat Completion
llama-3.1-sonar-large-128k-chat	70B	127,072	Chat Completion

Open-Source Models
Where possible, we try to match the Hugging Face implementation.
Model	Parameter Count	Context Length	Model Type
llama-3.1-8b-instruct	8B	131,072	Chat Completion
llama-3.1-70b-instruct	70B	131,072	Chat Completion

```

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics including a utility function to render text for display in Jupyter:

In [1]:
import os
import requests
from IPython.display import display, Markdown

def renderTextAsMarkdown(text: str):
    display(Markdown(text))
    
API_KEY = os.environ.get('PERPLEXITY_API_KEY')
DEFAULT_MODEL = 'llama-3.1-sonar-small-128k-online'
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = 'Be precise and concise.'
MID_LEVEL_DETAIL = 'Explain in a few sentences.'
GREAT_DETAIL = 'Explain in a lot of detail.'
POLITE_DETAIL = 'You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user.'
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": f"{direction}"
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    r = requests.post(url, json=payload, headers=headers)
    assert(r.status_code == 200)
    return r.json()

def renderResponse(d):
    return renderTextAsMarkdown(d.get("choices")[0].get("message").get("content"))

def getModel(d):
    return renderTextAsMarkdown(d.get("model"))
    
def getCitations(d):
    def formatReferences(url_list):
        return [f"[{i + 1}]: {url}" for i, url in enumerate(url_list)]
    for ref in formatReferences(d.get("citations")):
        display(ref)

Let's test the function on an example query using the default `llama-3.1-sonar-small-128k-online` model and then `llama-3.1-8b-instruct`.  Note the differences in response timing:

In [3]:
%%time

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
d = getPerplexityResponse(prompt)
renderResponse(d)

If you have a broken washing machine in Southampton, UK, you can consider the following options:

1. **Advanced Appliance Care**: They offer fixed-price repairs for all makes and models, with a focus on Southampton and Winchester areas. However, they do not provide same-day service when booking online[1].

2. **Domestec**: This service offers same/next day call outs and repairs all major brands of domestic appliances, including washing machines. They charge a £61.95+VAT one-off fixed call out & labour charge, with parts quoted separately[2].

3. **Hedge End Domestics**: A family-run business with over 35 years of experience, they provide same or next day service for various appliance repairs, including washing machines. They carry spare parts for major brands and offer competitive prices[3].

4. **AA Quality Domestics**: Experienced in fixing all makes and models of domestic appliances, they can usually visit within 24 hours and source spare parts quickly. They also offer new appliances if the old one is beyond repair[4].

5. **NAC (Domestic Appliances)**: Provides fast and reliable washing machine repairs in Southampton with same day or next day services available. Their engineers carry a wide range of parts and offer a guarantee on their work[5].

You can contact any of these services to book a repair, and they will send a qualified engineer to fix your washing machine.

CPU times: user 11.2 ms, sys: 4.71 ms, total: 15.9 ms
Wall time: 3.29 s


As of 09.11.24, Perplexity are offering citations as part of the API response.  We can pull them out as follows.  The URLs all work and point to valid companies that could help with this job:

In [4]:
getCitations(d)

'[1]: https://www.advancedappliancecare.co.uk/appliances-we-repair/washing-machine-repairs/'

'[2]: https://domestec.co.uk/southampton-appliance-repairs/'

'[3]: https://www.hedgeenddomestics.co.uk'

'[4]: https://www.aaqualitydomesticappliances.co.uk/domestic-appliance-repairs'

'[5]: https://nacrepair.co.uk/appliance-repairs/washing-machine-repair/washing-machine-repair-southampton/'

In [5]:
%%time

renderResponse(getPerplexityResponse(prompt, model='llama-3.1-8b-instruct'))

Sorry to hear that your washing machine is broken In the UK, you'll want to call a professional appliance repair service to fix your washing machine. You can search online for "washing machine repair Southampton" or "appliance repair Southampton" to find local companies that offer this service.

Typically, you'll want to contact a domestic appliance repair engineer, who is a skilled tradesperson trained to repair and maintain household appliances, including washing machines. They will assess the issue, provide a quote, and carry out the necessary repairs to get your washing machine working again.

You can also check online directories like Rated People, MyBuilder, or TrustATrader to find reputable appliance repair services in your area.

CPU times: user 6.25 ms, sys: 2.38 ms, total: 8.63 ms
Wall time: 1.09 s


In [6]:
%%time

renderResponse(getPerplexityResponse(prompt, model='llama-3.1-8b-instruct', direction=POLITE_DETAIL))

I'm so sorry to hear that your washing machine is broken Don't worry, I'm here to help you get it fixed.

In the UK, the trade you'll need to call is an appliance repair engineer, specifically a washing machine repair engineer. They specialize in fixing domestic appliances, including washing machines.

Here are a few options to get your washing machine repaired:

1. **Local Appliance Repair Services**: You can search online for appliance repair services in Southampton. Websites like Rated People, MyBuilder, or Checkatrade can connect you with local engineers who offer washing machine repair services. You can read reviews, check their ratings, and get quotes from multiple engineers.
2. **Manufacturer's Customer Service**: If your washing machine is still under warranty, you can contact the manufacturer's customer service department for assistance. They may be able to send a repair engineer or provide guidance on how to fix the issue.
3. **National Appliance Repair Companies**: Companies like AO Repair, Currys PC World Repair, or Bosch Home Appliances Repair offer washing machine repair services across the UK. You can visit their websites, enter your postcode, and book a repair appointment.
4. **Word of Mouth**: Ask friends, family, or neighbors if they know any reliable appliance repair engineers in Southampton. Personal recommendations can be a great way to find a trustworthy engineer.

Before calling an engineer, it's a good idea to:

* Check your warranty to see if it's still valid
* Gather information about the issue with your washing machine (e.g., error codes, symptoms)
* Have your washing machine's make and model ready

When you call the engineer, be prepared to provide your address, a brief description of the issue, and any relevant information about your washing machine.

I hope this helps you get your washing machine repaired quickly and efficiently

CPU times: user 7.28 ms, sys: 8.53 ms, total: 15.8 ms
Wall time: 2.47 s


In [7]:
%%time

renderResponse(getPerplexityResponse(prompt, model='llama-3.1-sonar-small-128k-online', direction=POLITE_DETAIL))

If you have a broken washing machine and live in Southampton, UK, there are several options to get it repaired. Here are a few steps you can take:

1. **Domestec Appliance Repairs**:
   - **Contact Information**: 02380 111 621
   - **Service**: Offers same/next day call outs for washing machine repairs in Southampton. They repair all major brands and provide a 12-month guarantee on parts and labor[1].

2. **AA Quality Domestics**:
   - **Contact Information**: 023 8049 1088
   - **Service**: Provides experienced tradesmen for washing machine repairs in Southampton and Winchester. They can usually visit within 24 hours and offer a quick repair service[2].

3. **Hedge End Domestics**:
   - **Contact Information**: 01489 782 072
   - **Service**: Offers same or next day appliance repair services, including washing machines, in Southampton and surrounding areas. They carry spare parts for major brands and have over 35 years of experience[3].

4. **NAC (Domestic Appliances)**:
   - **Contact Information**: Book online or call for details
   - **Service**: Provides fast and reliable washing machine repairs in Southampton with same or next day services available. They have a network of engineers who can diagnose and fix various types of washing machines[4].

5. **Advanced Appliance Care**:
   - **Contact Information**: Book online or call for details
   - **Service**: Offers fixed price repairs on all makes and models of washing machines in Southampton and Winchester. They have years of experience and good reviews[5].

You can choose any of these services based on your convenience and the specific needs of your washing machine. It's always a good idea to check their availability and any additional costs for parts before booking a repair.

CPU times: user 7.97 ms, sys: 5.37 ms, total: 13.3 ms
Wall time: 4.29 s


Let's see how it handles a query with a more detailed response:

In [8]:
%%time

prompt = """"I have a broken garden gate which is no longer closing properly.  
What trade do I need to call to fix it?"""
renderResponse(getPerplexityResponse(prompt, model='llama-3.1-sonar-small-128k-online', direction=GREAT_DETAIL))

To fix a broken garden gate that is no longer closing properly, you typically need to call a carpenter or a handyman who specializes in gate repair. Here’s a detailed breakdown of the steps you can take:

1. **Assess the Damage**: Before calling a professional, try to identify the issue with your gate. Common problems include:
   - Sagging or uneven gate alignment
   - Broken hinges or latches
   - Loose screws or bolts
   - Rotting wood

2. **DIY Repair**: If the issue is minor, you might be able to fix it yourself. For example, if the gate is sagging, you can try adjusting the hinges or adding spacers to ensure proper alignment, as described in the DIY video from YouTube[2].

3. **Call a Professional**: If the problem is more complex or if you're not comfortable with DIY repairs, it's best to hire a professional. Here are some options:
   - **Carpenter**: A carpenter specializes in woodworking and can repair or replace wooden gates, including fixing hinges, latches, and ensuring proper alignment.
   - **Handyman**: A handyman is a general repair person who can handle a variety of tasks, including gate repairs. They might not specialize in carpentry but can still fix many common issues.
   - **Fence/Gate Repair Specialist**: Some companies specialize in fence and gate repairs. These professionals have the expertise to handle all types of gate issues, from simple adjustments to complex repairs.

4. **Local Service Providers**: Websites like Angi can help you find local professionals who specialize in fence and gate repairs. You can read reviews and compare prices to find the best service for your needs[5].

5. **Cost Estimates**: The cost to repair a gate can vary widely depending on the extent of the damage and the materials needed. On average, fence gate repair costs range from $100 to $400[3].

By following these steps, you should be able to find the right professional to fix your broken garden gate and ensure it functions properly again.

CPU times: user 8.46 ms, sys: 9.91 ms, total: 18.4 ms
Wall time: 4.61 s


## 3. Streaming interface

This is an outline of how to build a streaming interface for Perplexity.  More work needs to be done to flesh it out.

In [9]:
%%time

def getPerplexityStreamingResponse(question, model=DEFAULT_MODEL, direction=MID_LEVEL_DETAIL, max_tokens=DEFAULT_TOKENS, temperature=DEFAULT_TEMP):
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": True,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": direction,
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    print('start')
    response_stream = requests.post(url, json=payload, headers=headers)
    print('end')
    for response in response_stream:
        print(len(response))
    #return response.get("choices")[0].get("message").get("content")

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
#getPerplexityStreamingResponse(prompt, direction=GREAT_DETAIL)

CPU times: user 5 μs, sys: 3 μs, total: 8 μs
Wall time: 12.2 μs
